In [1]:
import h5py
import numpy as np
import dask, dask.array
import pandas as pd
from dask.diagnostics import ProgressBar
import xarray as xr
import cftime
from pathlib import Path
import ptolemy
import pandas_indexing.accessors
from pandas_indexing import isin, ismatch, set_openscm_registry_as_default

In [2]:
def read_var(dataset, coords):
    return xr.DataArray(
        dask.array.from_delayed(
            dask.delayed(np.asarray)(dataset), shape=dataset.shape, dtype=dataset.dtype
        ),
        coords=coords,
    )

In [3]:
def read_coords(file):
    return [
        pd.Index(file["lat"][:, 0], name="lat"),
        pd.Index(file["lon"][0, :], name="lon"),
    ]

In [4]:
def concat_group(group, concat_dim, coords, sep=None):
    names = list(group)
    da = xr.concat(
        [read_var(group[n], coords) for n in names],
        dim=pd.Index(names, name=concat_dim),
    )
    if sep is None:
        return da

    return da.assign_coords(
        {
            concat_dim: da.indexes[concat_dim]
            .str.split(sep, expand=True)
            .rename(concat_dim.split(sep))
        }
    ).unstack(concat_dim)

In [5]:
def read_monthly(group, coords):
    months = list(group)
    month_ds = []
    for month in months:
        if "partitioning" in group[month]:
            partition = concat_group(
                group[month]["partitioning"], "var_sector", coords, sep="_"
            )
            sectoral_vars = frozenset(partition.indexes["var"])
        else:
            sectoral_vars = frozenset()

        monthly_das = {}
        for name in group[month]:
            h5ds = group[month][name]
            if not isinstance(h5ds, h5py.Dataset):
                continue

            da = read_var(h5ds, coords)
            if name in sectoral_vars:
                da = da * partition.sel(var=name, drop=True)
            monthly_das[name] = da
       
        month_ds.append(xr.Dataset(monthly_das))

    return xr.concat(month_ds, dim=pd.Index(months, name="month").astype(int))


In [6]:
def month_to_cftime(ds, year):
    return (
        ds.assign_coords(
            time=(
                "month",
                xr.CFTimeIndex(
                    [cftime.datetime(year, m, 1) for m in ds.indexes["month"]]
                ),
            )
        )
        .swap_dims(month="time")
        .drop("month")
    )

In [7]:
def read_year(filename):
     file = h5py.File(filename)
     coords = read_coords(file)
     parts = [
          #read_monthly(file["biosphere"], coords),
          #read_monthly(file["burned_area"], coords),
          read_monthly(file["emissions"], coords)
     ]

     year = int(Path(filename).stem.split("_")[1])
     ds = month_to_cftime(xr.merge(parts), year)
     
     return ds

In [8]:
base_path = Path(
    "/Users/coroa/Library/CloudStorage/OneDrive-SharedLibraries-IIASA/RESCUE - WP 1/data"
)
path = base_path / "historical/rescue/gfed/data_raw/hdf5"
voc_unit = "kg VOC/yr" # or kg C/yr

# Emissions

In [9]:
emissions = xr.concat(
    [
        read_year(filename)
        for filename in sorted(path.glob("*.hdf5"), key=lambda p: p.stem)
    ],
    dim="time",
)

In [10]:
emissions

<xarray.Dataset>
Dimensions:              (lat: 720, lon: 1440, sector: 6, time: 312)
Coordinates:
  * lat                  (lat) float32 89.88 89.62 89.38 ... -89.62 -89.88
  * lon                  (lon) float32 -179.9 -179.6 -179.4 ... 179.6 179.9
  * sector               (sector) object 'AGRI' 'BORF' 'DEFO' ... 'SAVA' 'TEMF'
  * time                 (time) object 1997-01-01 00:00:00 ... 2022-12-01 00:...
Data variables:
    C                    (time, lat, lon, sector) float32 dask.array<chunksize=(1, 720, 1440, 6), meta=np.ndarray>
    DM                   (time, lat, lon, sector) float32 dask.array<chunksize=(1, 720, 1440, 1), meta=np.ndarray>
    small_fire_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>

In [11]:
def read_cell_area(filename):
    file = h5py.File(filename)
    coords = read_coords(file)
    return read_var(file["ancill"]["grid_cell_area"], coords).assign_attrs(unit="m2")


In [12]:
emissions["cell_area"] = read_cell_area(next(iter(path.glob("*.hdf5"))))

In [13]:
emissions["DM"].attrs.update(dict(unit="kg DM m-2 / month"))
emissions["C"].attrs.update(dict(unit="g C m-2 / month"))

In [14]:
# with ProgressBar():
#     emissions[["DM", "cell_area"]].to_netcdf("/Users/coroa/test4.nc", encoding={v: dict(zlib=True) for v in ["DM", "cell_area"]})

In [15]:
# -----------------------------------------------#
# Get emissions factor for different species
# -----------------------------------------------#
#
file = base_path / "historical/rescue/gfed/data_raw/GFED4_Emission_Factors.txt"

_, marker, *sectors = pd.read_csv(
    file, sep="\s+", skiprows=15, nrows=1, header=None
).iloc[0]
assert (
    marker == "SPECIE"
), f"header in {file} is not in line 16 anymore or looks different"

ef = pd.read_csv(file, sep="\s+", header=None, names=sectors, comment="#").rename_axis(
    index="em", columns="sector"
)


# Aggregate NMVOC to a total in terms of kgC
nmvoc_species = pd.read_excel(base_path / "historical/rescue/gfed" / "NMVOC-species.xlsx", index_col=0)
nmvoc_species = nmvoc_species.set_axis(
    nmvoc_species.index.str.replace(r"^(\w+) *\(.*\)$", r"\1", regex=True).rename("em")
)

# Whether to convert to kgC
if voc_unit == "kg C/yr":
    nmvoc_factors = (
        nmvoc_species.loc[nmvoc_species["NMVOC?"].isin(["y"])]
        .pipe(lambda df: df["Carbon weight"] / df["Molecular weight"])
        .astype(float)
    )
elif voc_unit == "kg VOC/yr":
    nmvoc_factors = pd.Series(
        1, nmvoc_species.index[nmvoc_species["NMVOC?"].isin(["y"])]
    )
else:
    raise NotImplementedError(f"voc_unit must be 'kg VOC' or 'kg C', not: '{voc_unit}'")

ef.loc["NMVOC"] = ef.multiply(nmvoc_factors, axis=0).sum()

ef_per_DM = (
    ef.loc[["BC", "CH4", "CO", "CO2", "NH3", "NMVOC", "NOx", "OC", "SO2"]]
    / ef.loc["DM"]
)
# in kg {species} / kg DM

# Aggregate to countries

In [16]:
idxr = xr.open_dataarray(base_path / 'gridding_process_files/iso_mask.nc', chunks={'iso': 1})
idxr

<xarray.DataArray (iso: 237, lat: 280, lon: 720)>
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(237, 280, 720), dtype=float64, chunksize=(1, 280, 720), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -55.75 -55.25 -54.75 -54.25 ... 82.75 83.25 83.75
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * iso      (iso) object 'abw' 'afg' 'ago' 'aia' ... 'yem' 'zaf' 'zmb' 'zwe'

In [17]:
with xr.open_dataset(base_path / 'iam_files/cmip6/REMIND-MAGPIE_SSP5-34-OS/BC-em-openburning_input4MIPs_emissions_CMIP_REMIND-MAGPIE-SSP5-34-OS-V1_gn_201501-210012.nc') as template:
    dm_regrid = emissions["DM"].interp(lon=template.lon, lat=template.lat, method="linear")

In [18]:
cell_area = xr.DataArray(ptolemy.cell_area(lats=dm_regrid.lat, lons=dm_regrid.lon), attrs=dict(unit="m2"))

In [19]:
country_emissions = (
    (dm_regrid.groupby("time.year").sum() * cell_area * idxr)
    .sum(["lat", "lon"])
    .assign_attrs(dict(unit="kg DM / a"))
    * xr.DataArray(ef_per_DM)
).compute()

/Users/coroa/.local/conda/envs/concordia/lib/python3.11/site-packages/dask/array/core.py:4832: PerformanceWarning: Increasing number of chunks by factor of 156
  result = blockwise(


In [20]:
units = pd.MultiIndex.from_tuples(
    [
        ("BC", "kg BC/yr"),
        ("OC", "kg OC/yr"),
        ("CO", "kg CO/yr"),
        ("CO2", "kg CO2/yr"),
        ("CH4", "kg CH4/yr"),
        ("NH3", "kg NH3/yr"),
        ("NOx", "kg NOx/yr"),
        ("NMVOC", voc_unit),
        ("SO2", "kg SO2/yr"),
    ],
    names=["em", "unit"],
)

In [21]:
ur = set_openscm_registry_as_default()

In [22]:
emissions_df = (
    country_emissions.to_series()
    .unstack("year")
    .rename_axis(index={"iso": "country"})
    .pix.semijoin(units, how="left")
    .pix.convert_unit(lambda u: u.replace("kg", "kt"))
)
emissions_df.to_csv(base_path / "historical" / "rescue" / "gfed" / "LUC_emissions_v2.0.csv")

In [23]:
emissions_df.groupby(["em", "sector", "unit"]).sum().loc[:, 2013:2020]

year                            2013          2014          2015   
em    sector unit                                                  
BC    AGRI   kt BC/yr   1.598474e+02  2.030049e+02  2.120185e+02  \
      BORF   kt BC/yr   1.750057e+02  2.127689e+02  2.224507e+02   
      DEFO   kt BC/yr   2.129801e+02  3.169519e+02  4.475046e+02   
      PEAT   kt BC/yr   2.399710e+00  7.996606e+00  1.495902e+01   
      SAVA   kt BC/yr   9.329516e+02  9.507934e+02  9.615453e+02   
      TEMF   kt BC/yr   4.503524e+01  5.258891e+01  4.634975e+01   
CH4   AGRI   kt CH4/yr  1.240416e+03  1.575318e+03  1.645264e+03   
      BORF   kt CH4/yr  2.086068e+03  2.536205e+03  2.651612e+03   
      DEFO   kt CH4/yr  2.076556e+03  3.090281e+03  4.363170e+03   
      PEAT   kt CH4/yr  1.247849e+03  4.158235e+03  7.778690e+03   
      SAVA   kt CH4/yr  4.891692e+03  4.985241e+03  5.041616e+03   
      TEMF   kt CH4/yr  3.026368e+02  3.533975e+02  3.114703e+02   
CO    AGRI   kt CO/yr   2.173924e+04  2.760867e+04  2.883452e+04   
      BORF   kt CO/yr   4.445145e+04  5.404330e+04  5.650248e+04   
      DEFO   kt CO/yr   3.809068e+04  5.668563e+04  8.003447e+04   
      PEAT   kt CO/yr   1.259848e+04  4.198218e+04  7.853485e+04   
      SAVA   kt CO/yr   1.588539e+05  1.618918e+05  1.637226e+05   
      TEMF   kt CO/yr   7.926202e+03  9.255648e+03  8.157555e+03   
CO2   AGRI   kt CO2/yr  3.378108e+05  4.290170e+05  4.480658e+05   
      BORF   kt CO2/yr  5.211670e+05  6.336258e+05  6.624582e+05   
      DEFO   kt CO2/yr  6.729353e+05  1.001446e+06  1.413942e+06   
      PEAT   kt CO2/yr  1.021677e+05  3.404555e+05  6.368802e+05   
      SAVA   kt CO2/yr  4.251233e+06  4.332534e+06  4.381528e+06   
      TEMF   kt CO2/yr  1.483461e+05  1.732279e+05  1.526761e+05   
NH3   AGRI   kt NH3/yr  4.624918e+02  5.873608e+02  6.134402e+02   
      BORF   kt NH3/yr  9.520311e+02  1.157463e+03  1.210132e+03   
      DEFO   kt NH3/yr  5.447376e+02  8.106654e+02  1.144579e+03   
      PEAT   kt NH3/yr  7.979037e+01  2.658871e+02  4.973874e+02   
      SAVA   kt NH3/yr  1.311175e+03  1.336250e+03  1.351361e+03   
      TEMF   kt NH3/yr  7.565920e+01  8.834937e+01  7.786757e+01   
NMVOC AGRI   kt VOC/yr  4.567010e+03  5.800066e+03  6.057595e+03   
      BORF   kt VOC/yr  8.465604e+03  1.029233e+04  1.076067e+04   
      DEFO   kt VOC/yr  6.947801e+03  1.033955e+04  1.459842e+04   
      PEAT   kt VOC/yr  2.824660e+03  9.412675e+03  1.760802e+04   
      SAVA   kt VOC/yr  2.826162e+04  2.880210e+04  2.912781e+04   
      TEMF   kt VOC/yr  1.465807e+03  1.711664e+03  1.508592e+03   
NOx   AGRI   kt NOx/yr  6.628338e+02  8.417936e+02  8.791701e+02   
      BORF   kt NOx/yr  3.150103e+02  3.829840e+02  4.004112e+02   
      DEFO   kt NOx/yr  1.044422e+03  1.554283e+03  2.194494e+03   
      PEAT   kt NOx/yr  5.999276e+01  1.999151e+02  3.739755e+02   
      SAVA   kt NOx/yr  9.833814e+03  1.002188e+04  1.013521e+04   
      TEMF   kt NOx/yr  1.729353e+02  2.019414e+02  1.779830e+02   
OC    AGRI   kt OC/yr   4.901986e+02  6.225483e+02  6.501901e+02   
      BORF   kt OC/yr   3.360110e+03  4.085163e+03  4.271053e+03   
      DEFO   kt OC/yr   1.929108e+03  2.870853e+03  4.053359e+03   
      PEAT   kt OC/yr   3.611564e+02  1.203489e+03  2.251332e+03   
      SAVA   kt OC/yr   6.606306e+03  6.732645e+03  6.808780e+03   
      TEMF   kt OC/yr   8.646766e+02  1.009707e+03  8.899151e+02   
SO2   AGRI   kt SO2/yr  8.525194e+01  1.082693e+02  1.130765e+02   
      BORF   kt SO2/yr  3.850126e+02  4.680916e+02  4.893915e+02   
      DEFO   kt SO2/yr  1.638309e+02  2.438091e+02  3.442343e+02   
      PEAT   kt SO2/yr  2.399710e+01  7.996606e+01  1.495902e+02   
      SAVA   kt SO2/yr  1.210316e+03  1.233462e+03  1.247410e+03   
      TEMF   kt SO2/yr  9.907752e+01  1.156956e+02  1.019694e+02   

year                            2016          2017          2018   
em    sector unit                                                  
BC    AGRI   kt BC/yr   1.746621e+02  1.760193e+02  1.5